# Машинное обучение, ФКН ВШЭ

## Практическое задание 3. Градиентный спуск своими руками

### Общая информация
Дата выдачи: 05.10.2019

Мягкий дедлайн: 07:59MSK 14.10.2019 (за каждый день просрочки снимается 1 балл)

Жесткий дедлайн: 23:59MSK 16.10.2019

### О задании

В данном задании необходимо реализовать обучение линейной регрессии с помощью различных вариантов градиентного спуска.


### Оценивание и штрафы
Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимально допустимая оценка за работу — 10 баллов.

Сдавать задание после указанного срока сдачи нельзя. При выставлении неполного балла за задание в связи с наличием ошибок на усмотрение проверяющего предусмотрена возможность исправить работу на указанных в ответном письме условиях.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов (подробнее о плагиате см. на странице курса). Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

Неэффективная реализация кода может негативно отразиться на оценке.

Все ответы должны сопровождаться кодом или комментариями о том, как они были получены.


### Формат сдачи
Задания сдаются через систему Anytask. Инвайт можно найти на странице курса. Присылать необходимо ноутбук с выполненным заданием. 

Для удобства проверки самостоятельно посчитайте свою максимальную оценку (исходя из набора решенных задач) и укажите ниже.

**Оценка**: ...

## Реализация градиентного спуска

Реализуйте линейную регрессию с функцией потерь MSE, обучаемую с помощью:

** Задание 1 (1 балл)** Градиентного спуска;

** Задание 2 (1.5 балла)** Стохастического градиентного спуска;

** Задание 3 (2.5 балла)** Метода Momentum.


Во всех пунктах необходимо соблюдать следующие условия:

* Все вычисления должны быть векторизованы;
* Циклы средствами python допускается использовать только для итераций градиентного спуска;
* В качестве критерия останова необходимо использовать (одновременно):

    * проверку на евклидовую норму разности весов на двух соседних итерациях (например, меньше некоторого малого числа порядка $10^{-6}$, задаваемого параметром `tolerance`);
    * достижение максимального числа итераций (например, 10000, задаваемого параметром `max_iter`).
* Чтобы проследить, что оптимизационный процесс действительно сходится, будем использовать атрибут класса `loss_history` — в нём после вызова метода `fit` должны содержаться значения функции потерь для всех итераций, начиная с первой (до совершения первого шага по антиградиенту);
* Инициализировать веса можно случайным образом или нулевым вектором. 


Ниже приведён шаблон класса, который должен содержать код реализации каждого из методов.

In [21]:
import numpy as np
from sklearn.base import BaseEstimator

class LinearReg(BaseEstimator):
    def __init__(self, gd_type='stochastic', 
                 tolerance=1e-4, max_iter=1000, w0=None,
                 alpha=1e-3, eta=1e-2):
        """
        gd_type: 'full' or 'stochastic' or 'momentum'
        tolerance: for stopping gradient descent
        max_iter: maximum number of steps in gradient descent
        w0: np.array of shape (d) - init weights
        eta: learning rate
        alpha: momentum coefficient
        """
        self.gd_type = gd_type
        self.tolerance = tolerance
        self.max_iter = max_iter
        self.w0 = w0
        self.alpha = alpha
        self.w = None
        self.eta = eta
        self.h = 0 # for momentum method
        self.loss_history = None # list of loss function values at each training iteration
    
    def fit(self, X, y):
        """
        X: np.array of shape (ell, d)
        y: np.array of shape (ell)
        ---
        output: self
        """
        X = X.values
        y = y.values
        self.loss_history = []
        if not self.w0:
            self.w0 = np.zeros(X.shape[1])
        self.w = self.w0.copy()
        
        for step in range(1, self.max_iter):
            last_w = self.w.copy()
            self.w = self.calc_gradient(X, y, step)
            self.loss_history.append(self.calc_loss(X, y))
            if np.linalg.norm(last_w - self.w) < self.tolerance:
                break
        # print(step)
        return self
    
    def predict(self, X):
        if self.w is None:
            raise Exception('Not trained yet')

        return np.dot(X, self.w)
    
    def calc_gradient(self, X, y, step):
        """
        X: np.array of shape (ell, d) (ell can be equal to 1 if stochastic)
        y: np.array of shape (ell)
        ---
        output: np.array of shape (d)
        """
        step_size = self.eta/step
        if self.gd_type == 'full':
            result = self.w - 2 * step_size * np.dot(X.T, self.predict(X) - y) / y.shape[0]
        elif self.gd_type == 'stochastic':
            sample = np.random.randint(X.shape[0], size=10)
            result = self.w - 2 * step_size * np.dot(X[sample].T, self.predict(X[sample]) - y[sample]) / y.shape[0]
        elif self.gd_type == 'momentum':
            self.h = self.alpha * self.h + 2 * step_size * np.dot(X.T, self.predict(X) - y) / y.shape[0]
            result = self.w - self.h
        return result

    def calc_loss(self, X, y):
        """
        X: np.array of shape (ell, d)
        y: np.array of shape (ell)
        ---
        output: float 
        """ 
        return np.square(self.predict(X) - y).mean()
    
    def r2(self, X, y):
        return 1 - np.square(self.predict(X) - y).sum() / np.square(y - y.mean()).sum()

** Задание 4 (0 баллов)**. 
* Загрузите данные из домашнего задания 2 ([train.csv](https://www.kaggle.com/c/nyc-taxi-trip-duration/data));
* Разбейте выборку на обучающую и тестовую в отношении 7:3 с random_seed=0;
* Преобразуйте целевую переменную `trip_duration` как $\hat{y} = \log{(y + 1)}$.

In [2]:
import pandas as pd
import datetime
from sklearn.model_selection import train_test_split

data = pd.read_csv("../hw2/nyc-taxi-trip-duration/train.csv")

y = np.log1p(data['trip_duration'])
data['pickup_datetime'] = pd.to_datetime(data['pickup_datetime'], format='%Y-%m-%d %H:%M:%S')
data['month'] = data['pickup_datetime'].dt.month
data['hour'] = data['pickup_datetime'].dt.hour
data['weekday'] = data['pickup_datetime'].dt.weekday
X = data.drop(['trip_duration', 'id', 'pickup_datetime', 'store_and_fwd_flag', 'dropoff_datetime'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

** Задание 5 (3 балла)**. Обучите и провалидируйте модели на данных из предыдущего пункта, сравните качество между методами по метрикам MSE и $R^2$. Исследуйте влияние параметров `max_iter` и `alpha` на процесс оптимизации. Согласуется ли оно с вашими ожиданиями?

In [13]:
optimal_consts = [
    {
        'method': 'full',
        'alpha': 1e-3,
        'eta': 1e-3
    }, 
    {
        'method': 'stochastic',
        'alpha': 1e-3,
        'eta': 5
    }, 
    {
        'method': 'momentum',
        'alpha': 1e-2,
        'eta': 1e-3
    }
]
consts = [
    {
        'method': 'full',
        'alpha': [1e-3],
        'eta': [1e-1, 1e-2, 1e-3],
        'max_iter': [10, 20, 50]
    }, 
    {
        'method': 'stochastic',
        'alpha': [1e-3],
        'eta': [1, 5, 10],
        'max_iter': [10, 20, 50]
    }, 
    {
        'method': 'momentum',
        'alpha': [1e-1, 1e-2, 1e-3],
        'eta': [1e-1, 1e-2, 1e-3],
        'max_iter': [10, 20, 50]
    }
]

In [22]:
for const in optimal_consts:
    model = LinearReg(const['method'], eta=const['eta'], alpha=const['alpha'])
    model.fit(X_train, y_train)
    print('MSE = %s ; for method = %s' % (model.calc_loss(X_test, y_test), const['method']))
    print('R^2 = %s ; for method = %s' % (model.r2(X_test, y_test), const['method']))
    print('--------')

MSE = 0.6295202708386565 ; for method = full
R^2 = 0.000203236329585188 ; for method = full
--------
MSE = 0.6445186051835512 ; for method = stochastic
R^2 = -0.023616943628235187 ; for method = stochastic
--------
MSE = 0.6295176590959156 ; for method = momentum
R^2 = 0.00020738426899369333 ; for method = momentum
--------


In [14]:
for const in consts:
    for eta in const['eta']:
        for alpha in const['alpha']:
            for max_iter in const['max_iter']:
                model = LinearReg(const['method'], eta=eta, alpha=alpha, max_iter=max_iter)
                model.fit(X_train, y_train)
                print('MSE = %s ; for method = %s ; eta = %s ; alpha = %s; max_iter = %s' % (
                    model.calc_loss(X_test, y_test), const['method'], eta, alpha, max_iter))
    print('--------')

MSE = 6.304747848479411e+52 ; for method = full ; eta = 0.1 ; alpha = 0.001; max_iter = 10
MSE = 8.713687171967486e+98 ; for method = full ; eta = 0.1 ; alpha = 0.001; max_iter = 20
MSE = 8.593111901282663e+214 ; for method = full ; eta = 0.1 ; alpha = 0.001; max_iter = 50
MSE = 4.750013952330409e+34 ; for method = full ; eta = 0.01 ; alpha = 0.001; max_iter = 10
MSE = 2.5950374014481782e+60 ; for method = full ; eta = 0.01 ; alpha = 0.001; max_iter = 20
MSE = 2.5105229681751396e+113 ; for method = full ; eta = 0.01 ; alpha = 0.001; max_iter = 50
MSE = 1929618807949671.2 ; for method = full ; eta = 0.001 ; alpha = 0.001; max_iter = 10
MSE = 1811461295544736.8 ; for method = full ; eta = 0.001 ; alpha = 0.001; max_iter = 20
MSE = 0.6295202708386565 ; for method = full ; eta = 0.001 ; alpha = 0.001; max_iter = 50
--------
MSE = 7.78711612588528 ; for method = stochastic ; eta = 1 ; alpha = 0.001; max_iter = 10
MSE = 5.440788933700884 ; for method = stochastic ; eta = 1 ; alpha = 0.001; m

**Параметр eta согласуется с ожиданием. Чем больше этот параметр, чем больше вероятность, что мы перескочим минимум и убежим на бесконечность. Аналогично с параметром alpha. Чем он больше, тем у нас будет больше инерция.**

** Задание 6 (2 балла)**. Постройте графики (на одной и той же картинке) зависимости величины функции потерь от номера итерации для полного, стохастического градиентного спусков, а также для полного градиентного спуска с методом Momentum. Сделайте выводы о скорости сходимости различных модификаций градиентного спуска.

Не забывайте о том, что должны получиться *красивые* графики!

In [ ]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

### Бонус 

** Задание 7 (2 балла)**. Реализуйте линейную регрессию с функцией потерь MSE, обучаемую с помощью метода
[Adam](https://arxiv.org/pdf/1412.6980.pdf) - добавьте при необходимости параметры в класс модели, повторите пункты 5 и 6 и сравните результаты. 

** Задание 8 (2 балла)**. Реализуйте линейную регрессию с функцией потерь
$$ L(\hat{y}, y) = log(cosh(\hat{y} - y)),$$

обучаемую с помощью градиентного спуска.

** Задание 9 (0.01 балла)**.  Вставьте картинку с вашим любимым мемом в этот Jupyter Notebook